## 下載檔案

In [ ]:
! wget https://raw.githubusercontent.com/ywchiu/tibamedl/master/Data/zhiyu_face.zip
! wget https://raw.githubusercontent.com/ywchiu/tibamedl/master/Data/aragaki_face.zip
! wget https://raw.githubusercontent.com/ywchiu/tibamedl/master/Data/puff_face.zip

## 解壓縮檔案

In [ ]:
! unzip zhiyu_face.zip
! unzip puff_face.zip
! unzip aragaki_face.zip

## 建立訓練與測試資料集

In [ ]:
! rm -rf train
! rm -rf test

import os

if not os.path.exists('train/'):
    os.mkdir('train/')

if not os.path.exists('test/'):
    os.mkdir('test/')

In [ ]:
from sklearn.model_selection import train_test_split

def splitDataset(dirs):
    dataset = list(os.listdir(dirs))
    train_data, test_data = train_test_split(dataset, test_size= 0.2, random_state = 42)
    
    if not os.path.exists('train/'+dirs):
        os.mkdir('train/'+dirs)

    if not os.path.exists('test/'+dirs):
        os.mkdir('test/'+dirs)

    for f in train_data:
        os.rename(dirs + f, 'train/'+ dirs + f)

    for f in test_data:
        os.rename(dirs + f, 'test/' + dirs +f)


In [ ]:
splitDataset('zhiyu_face/')
splitDataset('puff_face/')
splitDataset('aragaki_face/')

## 建立 CNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten,Dropout
# Initialising the CNN
classifier = Sequential()

# Convolution
classifier.add(Conv2D(filters=32, 
           kernel_size=(3, 3), 
           padding = 'same',
           input_shape = (64, 64, 3),
           activation = 'relu'))

# Max Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Convolution
classifier.add(Conv2D(filters=32, 
          kernel_size=(3, 3), 
          padding = 'same', 
          activation = 'relu'))

# Max Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Flattening
classifier.add(Flatten())

# Fully Connected
classifier.add(Dense(units = 128, activation = 'relu')) 
classifier.add(Dropout(rate=0.1))  
classifier.add(Dense(units = 3, activation = 'softmax'))

# Compile
classifier.compile(optimizer = 'adam', 
          loss ='categorical_crossentropy', 
          metrics = ['accuracy'])

## 資料增幅

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(
    'train/', target_size = (64, 64),
     batch_size = 32,
     shuffle=True,
     class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(
    'test/', target_size = (64, 64),
    class_mode = 'categorical')

## 訓練模型

In [ ]:
history = classifier.fit(training_set,
            epochs=10,
            verbose = 1,
            validation_data = test_set)

## 單張預測

In [ ]:
import requests
res = requests.get('https://upload.wikimedia.org/wikipedia/commons/thumb/c/cd/Chou_Tzuyu_at_the_Golden_Disc_Awards_2019.png/220px-Chou_Tzuyu_at_the_Golden_Disc_Awards_2019.png')
with open('zhiyu_test.png', 'wb') as f:
    f.write(res.content)

In [ ]:
import face_recognition
from PIL import Image

image = face_recognition.load_image_file('zhiyu_test.png')
face_locations = face_recognition.face_locations(image)
top, right, bottom, left = face_locations[0]
faceImage = image[top:bottom, left:right]
final = Image.fromarray(faceImage)
crpim   = final.resize((64,64))
crpim.save('predict_face.png')

In [ ]:
crpim

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('predict_face.png', target_size= (64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
classifier.predict_classes(test_image)

In [ ]:
training_set.class_indices

## 遷移學習 (Transfer Learning)

In [ ]:
! pip install keras_vggface

In [ ]:
from keras_vggface.vggface import VGGFace


vgg_model = VGGFace(include_top=False, input_shape=(64, 64, 3), pooling='avg')
vgg_model.trainable = False

In [ ]:
from tensorflow.keras.layers import BatchNormalization

# Initialising the CNN
classifier = Sequential()

# Add VGG16
classifier.add(vgg_model)

# Flattening
classifier.add(Flatten())

# Fully Connected
classifier.add(Dense(units = 512, activation = 'relu')) 
classifier.add(BatchNormalization())
classifier.add(Dropout(rate=0.1))  

classifier.add(Dense(units = 512, activation = 'relu')) 
classifier.add(BatchNormalization())
classifier.add(Dropout(rate=0.1))  

classifier.add(Dense(units = 3, activation = 'softmax'))

classifier.compile(optimizer = 'adam', 
          loss ='categorical_crossentropy', 
          metrics = ['accuracy'])

In [ ]:
classifier.summary()

## 模型訓練

In [ ]:
history = classifier.fit(training_set,
            epochs=100,
            verbose = 1,
            validation_data = test_set)